# Experimentos iniciales con LLMs que no hagan explotar a mi compu

La primera parte radica en explorar con distintas estrategias de prompting para poder encontrar los mejores resultados básicos sin ningún tipo de ajuste del modelo. Entre más sencillo sea el tipo de prompt mejor. La segunda sección corresponde a la implementación de PPO mediante HuggingFace.

In [1]:
import transformers, torch, datasets
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, Dataset

In [2]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

cuda
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 

In [3]:
print(f'Memoria actual: {torch.cuda.memory_allocated(device=dev)}')
print(f'Memoria máxima: {torch.cuda.max_memory_allocated(device=dev)}')
print(f'Memoria reservada: {torch.cuda.memory_reserved(device=dev)}')
print(f'Máxima memoria reservada: {torch.cuda.max_memory_reserved(device=dev)}')
print(f'CUDA Device name: {torch.cuda.get_device_name()}')

Memoria actual: 0
Memoria máxima: 0
Memoria reservada: 0
Máxima memoria reservada: 0
CUDA Device name: NVIDIA GeForce RTX 3060 Ti


## LLama 3 🦙

In [4]:
llama_id = 'meta-llama/Llama-3.2-1B'
llama_model = AutoModelForCausalLM.from_pretrained(llama_id).to(dev)
llama_tokenizer = AutoTokenizer.from_pretrained(llama_id)

In [5]:
def llama_gen(prompt, repetitions, llm_tokens):
    """
    Generación de respuestas de Llama.

    prompt = 'str' ; El prompt con la proposición lógica.
    repetitions = int ; Cantidad de iteraciones a obtener.
    llm_tokens = int ; Límite de tokens.
    """    
    print(f'EL PROMPT ES: {prompt}')
    print("----------------")
    for i in range(repetitions):
        llm_input = llama_tokenizer(prompt, return_tensors = 'pt').to(dev)
        input_length = llm_input.input_ids.shape[1]
        llm_gen_ids = llama_model.generate(**llm_input, max_new_tokens = llm_tokens)
        print(llama_tokenizer.batch_decode(llm_gen_ids[:, input_length:], skip_special_tokens = True)[0])
        print("----")

In [7]:
prop_log = 'If Mason left his job, then he will not receive any salary.'
log_prompt = f"""Write the following statement in terms of propositional logic. Statement: "{prop_log}" \n
A proposition is a singular statement that can be valuated true or false. Determine which propositions exist within the whole statement."""

#llama_gen(log_prompt, 10, 75)

In [9]:
more_context = f""" A logical proposition is like the following:
Q: If Daniel has a pet dog, then he will take it for a walk every day.

A proposition is a declaritve sentence that is either True or False.

A: The propositions from this statement are:
1 Daniel hast a pet dog.
2 He takes the dog for a walk once a day.

Q: "{prop_log}" 
A The propositions from this statement are:
"""

llama_gen(more_context, 10, 40)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


EL PROMPT ES:  A logical proposition is like the following:
Q: If Daniel has a pet dog, then he will take it for a walk every day.

A proposition is a declaritve sentence that is either True or False.

A: The propositions from this statement are:
1 Daniel hast a pet dog.
2 He takes the dog for a walk once a day.

Q: "If Mason left his job, then he will not receive any salary." 
A The propositions from this statement are:

----------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 Mason left his job.
2 He will not receive any salary.
3 If he left his job, then he will not receive any salary.
4 If he left his job, then he will not
----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

1 Mason left his job.
2 He will not receive any salary.

A: The propositions from this statement are:
1 Mason did not leave his job.
2 He will receive no salary.

A:
----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 Mason left his job.
2 He will not receive any salary.
3 He will not receive any salary if he leaves his job.
4 He will not receive any salary if he does not leave his
----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 Mason left his job.
2 He will not receive any salary.

Q: "If it is raining, then it will be raining cats and dogs." 
A The propositions from this statement are:

----
1 Mason left his job.
2 He will not receive any salary.

Q: "If it is raining, then I will take my umbrella." 
A The propositions from this statement are:
1 It
----


## PPO

In [10]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, apply_chat_template

In [11]:
modus_tollens = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\MSc-Thesis\Datasets\LogicBench\LogicBench(Aug)\propositional_logic\modus_tollens\data_instances.json')

In [12]:
prompt = []
for _ in modus_tollens['data_samples'][:5]:
    ds_sample = [{'role': 'user', 'content': str(_['context'])}]
    #print(ds_sample)
    prompt.append(ds_sample)

c1 = [{'role':'assitant', 'content': 'p = Mason left his job. q = Mason will recieve any salary.'}]
c2 = [{'role':'assitant', 'content': 'p = Daniel has a pet dog. q = Daniel will take the dog for a walk every day.'}]
c3 = [{'role':'assitant', 'content': 'p = Jack won the lottery. q = Dan will buy a house.'}]
c4 = [{'role':'assitant', 'content': 'p = Levi is studying for his exam. q = Levi will pass with flying colors.'}]
c5 = [{'role':'assitant', 'content': 'p = Levi has an exam tomorrow. q = Levi will stay up late to study.'}]

completition = [c1, c2, c3, c4, c5]
dataset_dict = {
    'prompt': prompt,
    'completition': completition
}

dataset = Dataset.from_dict(dataset_dict)
dataset
#dataset = dataset.map(apply_chat_template, fn_kwargs = {'tokenizer': llama_tokenizer})

Dataset({
    features: ['prompt', 'completition'],
    num_rows: 5
})

In [13]:
ppo_llama = AutoModelForCausalLMWithValueHead.from_pretrained(llama_id)
ppo_ref = AutoModelForCausalLMWithValueHead.from_pretrained(llama_id)
llama_tokenizer.pad_token = llama_tokenizer.eos_token

In [14]:
# ¿QUÉ PONGO COMO REWARD MODEL?

ppo_config = {'mini_batch_size': 1, 'batch_size': 1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(
    config, 
    ppo_llama,
    ppo_ref,
    llama_tokenizer,
    #reward_model = ????????????????
    train_dataset = dataset
)

TypeError: __init__() missing 1 required positional argument: 'reward_model'